In [1]:
import sys.process._
%AddJar file:///home/user/Documents/RCP-216/lsa.jar

Starting download from file:///home/user/Documents/RCP-216/lsa.jar
Finished download of lsa.jar


In [2]:
import com.cloudera.datascience.lsa._
import com.cloudera.datascience.lsa.ParseWikipedia._
import com.cloudera.datascience.lsa.RunLSA._
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import breeze.linalg.{DenseMatrix => BDenseMatrix,
              DenseVector => BDenseVector, SparseVector => BSparseVector}

import breeze.linalg.{DenseMatrix=>BDenseMatrix, DenseVector=>BDenseVector, SparseVector=>BSparseVector}


In [3]:
val wikixmlfile = "/home/user/Documents/RCP-216/data/enwiki.xml"

wikixmlfile = /home/user/Documents/RCP-216/data/enwiki.xml


/home/user/Documents/RCP-216/data/enwiki.xml

In [4]:
val sampleSize = 0.3
val xmlPages = ParseWikipedia.readFile(wikixmlfile, sc).sample(false, sampleSize, 11L)
xmlPages.take(2)

sampleSize = 0.3
xmlPages = PartitionwiseSampledRDD[2] at sample at <console>:58


Array(<page>
    <title>AfghanistanPeople</title>
    <ns>0</ns>
    <id>15</id>
    <redirect title="Demographics of Afghanistan" />
    <revision>
      <id>616420354</id>
      <parentid>135089040</parentid>
      <timestamp>2014-07-10T19:18:43Z</timestamp>
      <contributor>
        <username>RussBot</username>
        <id>279219</id>
      </contributor>
      <minor />
      <comment>Robot: Fixing double redirect to [[Demographics of Afghanistan]]</comment>
      <model>wikitext</model>
      <format>text/x-wiki</format>
      <text xml:space="preserve">#REDIRECT [[Demographics of Afghanistan]] {{R from CamelCase}}</text>
      <sha1>em7...


[<page>
    <title>AfghanistanPeople</title>
    <ns>0</ns>
    <id>15</id>
    <redirect title="Demographics of Afghanistan" />
    <revision>
      <id>616420354</id>
      <parentid>135089040</parentid>
      <timestamp>2014-07-10T19:18:43Z</timestamp>
      <contributor>
        <username>RussBot</username>
        <id>279219</id>
      </contributor>
      <minor />
      <comment>Robot: Fixing double redirect to [[Demographics of Afghanistan]]</comment>
      <model>wikitext</model>
      <format>text/x-wiki</format>
      <text xml:space="preserve">#REDIRECT [[Demographics of Afghanistan]] {{R from CamelCase}}</text>
      <sha1>em7my23894czbu7ksvulil0g8l6pcrc</sha1>
    </revision>
  </page>, <page>
    <title>AfghanistanCommunications</title>
    <ns>0</ns>
    <id>18</id>
    <redirect title="Communications in Afghanistan" />
    <revision>
      <id>74466499</id>
      <parentid>15898952</parentid>
      <timestamp>2006-09-08T04:14:42Z</timestamp>
      <contributor>
       

In [6]:
val plainText = xmlPages.filter(_ != null).flatMap(ParseWikipedia.wikiXmlToPlainText)
plainText.take(2)

plainText = MapPartitionsRDD[4] at flatMap at <console>:60


Array((Autism,"Autism
Autism is a neurodevelopmental disorder characterized by impaired social interaction, verbal and non-verbal communication, and restricted and repetitive behavior. Parents usually notice signs in the first two years of their child's life.  The signs typically develop gradually, but some children with autism will reach their developmental milestones at a normal pace and then regress.
Autism is highly heritable, but the cause includes both environmental factors and genetic susceptibility.  In rare cases, autism is strongly associated with agents that cause birth defects.  Controversies surround other proposed environmental cau...


[(Autism,Autism
 
 
 
 
 
 
 
 
Autism is a neurodevelopmental disorder characterized by impaired social interaction, verbal and non-verbal communication, and restricted and repetitive behavior. Parents usually notice signs in the first two years of their child's life.  The signs typically develop gradually, but some children with autism will reach their developmental milestones at a normal pace and then regress.   
Autism is highly heritable, but the cause includes both environmental factors and genetic susceptibility.  In rare cases, autism is strongly associated with agents that cause birth defects.  Controversies surround other proposed environmental causes;  for example, the vaccine hypotheses are biologically implausible and have been disproven in scientific studies. The diagnostic criteria require that symptoms become apparent in early childhood, typically before age three.    Autism affects information processing in the brain by altering how nerve cells and their synapses conne

In [14]:
plainText.info()


Name: Compile Error
Message: <console>:63: error: value info is not a member of org.apache.spark.rdd.RDD[(String, String)]
       plainText.info()
                 ^

StackTrace: 

In [6]:
val stopWords = sc.broadcast(ParseWikipedia
                         .loadStopWords("/home/user/Documents/RCP-216/deps/lsa/src/main/resources/stopwords.txt")).value
val lemmatized = plainText.mapPartitions(iter => {
           val pipeline = ParseWikipedia.createNLPPipeline();
           iter.map{ case(title, contents) =>
                         (title, ParseWikipedia.plainTextToLemmas(contents, stopWords, pipeline))};
       }).cache()

lemmatized.take(2)
val filtered = lemmatized.filter(_._2.size > 1)

Adding annotator tokenize
TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
Adding annotator ssplit
edu.stanford.nlp.pipeline.AnnotatorImplementations:
Adding annotator pos
Reading POS tagger model from edu/stanford/nlp/models/pos-tagger/english-left3words/english-left3words-distsim.tagger ... done [1,1 sec].
Adding annotator lemma


stopWords = Set(down, it's, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, ours	ourselves, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, w...


Set(down, it's, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, ours	ourselves, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, with, from, you've, they've, what's, wouldn't, both, could, its, under, which, you'd, an, be, here's, into, where, he'll, her, themselves, were, more, we'd, where's, they're, who's, between, aren't, about, doesn't, how's, against, during, no, very, we, hav

In [7]:
val numTerms = 1000 // nombre de termes à garder (les plus fréquents)
val (termDocMatrix, termIds, docIds, idfs) =
            ParseWikipedia.termDocumentMatrix(filtered, stopWords, numTerms, sc)

Adding annotator tokenize
Adding annotator tokenize
Adding annotator ssplit
Adding annotator ssplit
Adding annotator pos
Adding annotator pos
Adding annotator lemma
Adding annotator lemma
Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma
[Stage 3:>                                                          (0 + 4) / 5]Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma
Number of terms: 1000                                                           


numTerms = 1000
termDocMatrix = MapPartitionsRDD[17] at map at ParseWikipedia.scala:64
termIds = Map(645 -> love, 892 -> identify, 69 -> court, 809 -> time, 629 -> house, 365 -> text, 138 -> size, 760 -> american, 101 -> agree, 479 -> russian, 347 -> christian, 846 -> relatively, 909 -> film, 333 -> earlier, 628 -> order, 249 -> friend, 893 -> rise, 518 -> change, 962 -> located, 468 -> museum, 234 -> higher, 941 -> distance, 0 -> rate, 777 -> debate, 555 -> port, 666 -> sound, 88 -> point, 481 -> probably, 352 -> demonstrate, 408 -> english, 977 -> obtain, 170 -> join, 523 -> response, 582 -> son, 762 -> characteristic, 115 -> show, 683 -> refer, 730 -> use, 217 -> problem, 276 -> bir...


Map(645 -> love, 892 -> identify, 69 -> court, 809 -> time, 629 -> house, 365 -> text, 138 -> size, 760 -> american, 101 -> agree, 479 -> russian, 347 -> christian, 846 -> relatively, 909 -> film, 333 -> earlier, 628 -> order, 249 -> friend, 893 -> rise, 518 -> change, 962 -> located, 468 -> museum, 234 -> higher, 941 -> distance, 0 -> rate, 777 -> debate, 555 -> port, 666 -> sound, 88 -> point, 481 -> probably, 352 -> demonstrate, 408 -> english, 977 -> obtain, 170 -> join, 523 -> response, 582 -> son, 762 -> characteristic, 115 -> show, 683 -> refer, 730 -> use, 217 -> problem, 276 -> birth, 994 -> decline, 308 -> britain, 741 -> rather, 5 -> reference, 873 -> see, 449 -> element, 120 -> student, 247 -> short, 379 -> describe, 878 -> limit, 440 -> object, 655 -> story, 511 -> travel, 614 -> territory, 269 -> previous, 677 -> association, 202 -> recent, 597 -> party, 861 -> wife, 10 -> entirely, 385 -> public, 384 -> computer, 56 -> website, 533 -> variety, 550 -> africa, 142 -> relat

In [8]:
termDocMatrix.cache()

MapPartitionsRDD[17] at map at ParseWikipedia.scala:64

In [9]:
val mat = new RowMatrix(termDocMatrix)
val k = 200 // nombre de valeurs singulières à garder
val svd = mat.computeSVD(k, computeU = true)

[Stage 7:===========>                                               (1 + 4) / 5]

mat = org.apache.spark.mllib.linalg.distributed.RowMatrix@66ec53b9
k = 200
svd = 


error: missing or invalid dependency detected while loading class file 'SequenceFileRDDFunctions.class'.
Could not access type AnyRef in package scala,
because it (or its dependencies) are missing. Check your build definition for
missing or conflicting dependencies. (Re-run with `-Ylog-classpath` to see the problematic classpath.)
A full rebuild may help if 'SequenceFileRDDFunctions.class' was compiled against an incompatible version of scala.
error: missing or invalid dependency detected while loading class file 'PartitionwiseSampledRDDPartition.class'.
Could not access type AnyRef in package scala,
because it (or its dependencies) are missing. Check your build definition for
missing or conflicting dependencies. (Re-run with `-Ylog-classpath` to see the problematic classpath.)
A full rebuild may help if 'PartitionwiseSampledRDDPartition.class' was compiled against an incompatible version of scala.
error: missing or invalid dependency detected while loading class file 'OrderedRDDFuncti

SingularValueDecomposition(org.apache.spark.mllib.linalg.distributed.RowMatrix@3173bb34,[0.9008745961921408,0.7226627218453527,0.6828734371832617,0.49253852830878964,0.4000303651661773,0.38169633705404293,0.3701455217441776,0.3609445048785556,0.3524121175002394,0.35150733385266625,0.3394174416269283,0.33697998077895985,0.3335173980982471,0.33226126088267355,0.32831993567812656,0.3215013205006916,0.3179538088214402,0.3139877126782244,0.3086667000223249,0.3058858342076883,0.3052098012693117,0.29751845010471617,0.2950397157104417,0.29418830202397134,0.2917346941001857,0.2903370603208323,0.27603101679095476,0.2727423029873338,0.2669647776982502,0.2640359990253202,0.26184514481916576,0.2558517321599271,0.253865527368309,0.24845161932929163,0.2464165612367011,0.24220222323403073,0.23800500993757595,0.2365464509737028,0.23263153209788975,0.23115389268816575,0.22678758580415312,0.2252379437826403,0.22462137921957379,0.21934057613909066,0.21780990449409568,0.2136153162798397,0.2101282703066504,

In [10]:
val topConceptTerms = RunLSA.topTermsInTopConcepts(svd, 10, 10, termIds)
val topConceptDocs = RunLSA.topDocsInTopConcepts(svd, 10, 10, docIds)
for ((terms, docs) <- topConceptTerms.zip(topConceptDocs)) {
          println("Concept terms: " + terms.map(_._1).mkString(", "));
          println("Concept docs: " + docs.map(_._1).mkString(", "));
          println();
       }

[Stage 407:>                                                        (0 + 0) / 5]Concept terms: toward, overall, heavily, entirely, unlike, hope, behind, occupy, poor, attention
Concept docs: Biathlon World Championships, Antipope Victor IV, Christopher Báthory, Alfonso IV, Aberdeen Bestiary, Cucurbitaceae, List of anthropologists, Alexius, Bactericide, Teochew cuisine

Concept terms: code, significantly, somewhat, unlike, possibly, powerful, toward, heavily, entirely, quickly
Concept docs: Currency code, Biathlon World Championships, List of Olympic medalists in biathlon, Antipope Victor IV, Alfonso IV, Aberdeen Bestiary, BLM, Cucurbitaceae, Christopher Báthory, List of anthropologists

Concept terms: language, game, american, film, government, population, music, computer, theory, character
Concept docs: David Bowman (Space Odyssey), Class, Character, Britain, Complexity theory, Brittonic, Demographics of Antigua and Barbuda, Creation, Bundesrat, Body

Concept terms: language, governme

topConceptTerms = ArrayBuffer(ArraySeq((toward,-6.77041324515501E-4), (overall,-7.086751602206204E-4), (heavily,-7.228581736882505E-4), (entirely,-7.288085561717311E-4), (unlike,-7.346935414119862E-4), (hope,-7.626646304719053E-4), (behind,-7.632949817612047E-4), (occupy,-7.669074995204223E-4), (poor,-7.679457738504068E-4), (attention,-7.787008887555238E-4)), ArraySeq((code,0.057052840952430836), (significantly,-0.0038598019802077816), (somewhat,-0.004351953605441758), (unlike,-0.0044189436979907536), (possibly,-0.004562194082433074), (powerful,-0.004636583511455492), (toward,-0.004640362394194138), (heavily,-0.004642479763188129), (entirely,-0.004659304445032707), (quickly,-0.004791883352648484)), ArraySeq((language,0.11783834885199357), (game,0.100783618069...


error: missing or invalid dependency detected while loading class file 'SparseVector.class'.
Could not access type AnyRef in package scala,
because it (or its dependencies) are missing. Check your build definition for
missing or conflicting dependencies. (Re-run with `-Ylog-classpath` to see the problematic classpath.)
A full rebuild may help if 'SparseVector.class' was compiled against an incompatible version of scala.


ArrayBuffer(ArraySeq((toward,-6.77041324515501E-4), (overall,-7.086751602206204E-4), (heavily,-7.228581736882505E-4), (entirely,-7.288085561717311E-4), (unlike,-7.346935414119862E-4), (hope,-7.626646304719053E-4), (behind,-7.632949817612047E-4), (occupy,-7.669074995204223E-4), (poor,-7.679457738504068E-4), (attention,-7.787008887555238E-4)), ArraySeq((code,0.057052840952430836), (significantly,-0.0038598019802077816), (somewhat,-0.004351953605441758), (unlike,-0.0044189436979907536), (possibly,-0.004562194082433074), (powerful,-0.004636583511455492), (toward,-0.004640362394194138), (heavily,-0.004642479763188129), (entirely,-0.004659304445032707), (quickly,-0.004791883352648484)), ArraySeq((language,0.11783834885199357), (game,0.10078361806902143), (american,0.09920535407503207), (film,0.09815083113529126), (government,0.08068082278361109), (population,0.08001281768330372), (music,0.07869741394428749), (computer,0.07837863011675318), (theory,0.07317611675322465), (character,0.071824801